# Модель детекции аномалий

## Загрузим и импортируем нужные модули


In [1]:
!pip install -q catboost

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


## Выгрузим наши датасеты

In [5]:
SEED = 42
PATH = '/kaggle/input/tractor-data/'

In [6]:
data_anomaly = pd.read_csv(PATH + "dataset._anomaly.csv", sep=";", decimal = ',')
data_normal = pd.read_csv(PATH + "dataset._normal.csv", sep=";", decimal = ',')
data_problems = pd.read_csv(PATH + "dataset._problems.csv", sep=";", decimal = ',')

/tmp/ipykernel_34/652216023.py:2: DtypeWarning: Columns (1,3,5,12,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,37,41,42,44) have mixed types. Specify dtype option on import or set low_memory=False.
  data_normal = pd.read_csv(PATH + "dataset._normal.csv", sep=";", decimal = ',')
/tmp/ipykernel_34/652216023.py:3: DtypeWarning: Columns (1,3,5,12,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,37,41,42,44) have mixed types. Specify dtype option on import or set low_memory=False.
  data_problems = pd.read_csv(PATH + "dataset._problems.csv", sep=";", decimal = ',')


## Произведем препроцессинг данных

In [7]:
features = [col for col in data_problems.columns if col not in ['Дата и время']]

In [8]:
data_normal.replace("-",np.nan,inplace=True)
data_normal.replace('        -',np.nan,inplace=True)
data_normal = data_normal[features].dropna(how='all')

data_problems.replace("-",np.nan,inplace=True)
data_problems.replace('        -',np.nan,inplace=True)
data_problems = data_problems[features].dropna(how='all')

data_anomaly.replace("-",np.nan,inplace=True)
data_anomaly.replace('        -',np.nan,inplace=True)
data_anomaly = data_anomaly[features].dropna(how='all')

data_normal['problem'] = np.array([0]*len(data_normal))
data_problems['problem'] = np.array([1]*len(data_problems))
data_anomaly['problem'] = np.array([0]*len(data_anomaly))

data_normal['anomaly'] = np.array([0]*len(data_normal))
data_problems['anomaly'] = np.array([0]*len(data_problems))
data_anomaly['anomaly'] = np.array([1]*len(data_anomaly))

/tmp/ipykernel_34/2537589906.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_normal.replace("-",np.nan,inplace=True)
/tmp/ipykernel_34/2537589906.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_normal.replace('        -',np.nan,inplace=True)
/tmp/ipykernel_34/2537589906.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('fut

In [9]:
metki = ['Нейтраль КПП (spn3843)', 'Стояночный тормоз (spn3842)',
       'Аварийная температура охлаждающей жидкости (spn3841)',
       'Засоренность воздушного фильтра (spn3840)',
       'Засоренность фильтра КПП (spn3847)',
       'Аварийное давление масла ДВС (spn3846)',
       'Засоренность фильтра ДВС (spn3845)',
       'Засоренность фильтра рулевого управления (spn3844)',
       'Засоренность фильтра навесного оборудования (spn3851)',
       'Недопустимый уровень масла в гидробаке (spn3850)',
       'Аварийная температура масла в гидросистеме (spn3849)',
       'Аварийное давление в I контуре тормозной системы (spn3848)',
       'Аварийное давление в II контуре тормозной системы (spn3855)',
       'Зарядка АКБ (spn3854)', 'Отопитель (spn3853)',
       'Выход блока управления двигателем (spn3852)',
       'Включение тормозков (spn3859)', 'Засоренность фильтра слива (spn3858)',
       'Аварийное давление масла КПП (spn3857)',
       'Аварийная температура масла ДВС(spn3856)',
       'Неисправность тормозной системы (spn3863)', 'Термостарт (spn3862)',
       'Разрешение запуска двигателя (spn3861)', 'Низкий уровень ОЖ (spn3860)',
       'Аварийная температура масла ГТР (spn3867)',
       'Необходимость сервисного обслуживания (spn3866)',
       'Подогрев топливного фильтра (spn3865)', 'Вода в топливе (spn3864)',
       'Холодный старт (spn3871)', 'Крутящий момент (spn513), Нм',
       'Положение рейки ТНВД (spn51), %', 'Расход топлива (spn183), л/ч',
       'ДВС. Температура наддувочного воздуха, °С',
       'Давление наддувочного воздуха двигателя (spn106), кПа',
       'Текущая передача (spn523)',
       'Температура масла гидравлики (spn5536), С', 'Педаль слива (spn598)']

In [10]:
def to_standart(x):
    try:
        return int(x)
    except:
        return 0

data_anomaly[metki] = data_anomaly[metki].map(lambda x: to_standart(x))
data_problems[metki] = data_problems[metki].map(lambda x: to_standart(x))
data_normal[metki] = data_normal[metki].map(lambda x: to_standart(x))

In [11]:
def to_time(x):
    try:
        return int(x[:3])*60+int(x[-2:])
    except:
        return np.nan
data_anomaly['Значение счетчика моточасов, час:мин'] = data_anomaly['Значение счетчика моточасов, час:мин'].map(lambda x: to_time(x))
data_problems['Значение счетчика моточасов, час:мин'] = data_problems['Значение счетчика моточасов, час:мин'].map(lambda x: to_time(x))
data_normal['Значение счетчика моточасов, час:мин'] = data_normal['Значение счетчика моточасов, час:мин'].map(lambda x: to_time(x))

In [12]:
num_feat = ['Полож.пед.акселер.,%', 'Нагрузка на двигатель, %',
       'Давл.масла двиг.,кПа', 'Темп.масла двиг.,°С', 'Обор.двиг.,об/мин',
       'Значение счетчика моточасов, час:мин', 
       'КПП. Температура масла', 'КПП. Давление масла в системе смазки',
       'Скорость', 'ДВС. Давление смазки',
       'ДВС. Температура охлаждающей жидкости',
       'Давление в пневмостистеме (spn46), кПа', 'Уровень топлива % (spn96)',
       'Электросистема. Напряжение', 'ДВС. Частота вращения коленчатого вала']
cat_feat = ['iButton2','Сост.пед.сцепл.']
features = num_feat+cat_feat
len(features)

17

In [13]:
data_anomaly[cat_feat] = data_anomaly[cat_feat].fillna('no_data')
data_problems[cat_feat] = data_problems[cat_feat].fillna('no_data')
data_normal[cat_feat] = data_normal[cat_feat].fillna('no_data')
data_anomaly[cat_feat] = data_anomaly[cat_feat].astype("category")
data_problems[cat_feat] = data_problems[cat_feat].astype("category")
data_normal[cat_feat] = data_normal[cat_feat].astype("category")

## Создание новых датасетов

- По анализу результатов алгоритмы t-SNE мы предположим, что внутри датасета с аномальными данными нет разделения на кластеры 
- Это может означать аномальность всех данных, имеющихся в dataset._anomaly
- Для обучения на детекцию аномальности воспользуемся новым датасетом, полученным в результате соединения датасета с нормальными значениями и аномальными (учитывая разметку)

In [14]:
df_anomaly = pd.concat([data_normal, data_anomaly])

In [15]:
for_rep = []
for feat in num_feat:
    try:
        df_anomaly[feat] = df_anomaly[feat].astype(float)
    except:
        for_rep.append(feat)

In [16]:
def to_int(x):
    return str(x).replace(',','.')
df_anomaly[for_rep] = df_anomaly[for_rep].map(lambda x: to_int(x))
df_problems[for_rep] = df_problems[for_rep].map(lambda x: to_int(x))
df_all[for_rep] = df_all[for_rep].map(lambda x: to_int(x))
df_ano_pro[for_rep] = df_ano_pro[for_rep].map(lambda x: to_int(x))

In [17]:
df_anomaly[num_feat] = df_anomaly[num_feat].astype("float64")

## Обучение

Мы попробуем детектировать аномальность, ориентируясь на информативные колонки, отображающие данные телеметрии

Для простоты воспользуемся относительно простой, быстрой и интепретируемой моделью машинного обучения - бустингом. Ее преимущетсвами явлются не только простота, но и точность. В купе с возможностью обучаться на больших объемах данных, легко масштабироваться и учитывать огромное количество факторов, она станет основной моделью, используемой для наших предсказаний.

### Признаки, на которых будем обучаться

In [45]:
num_feat = ['Полож.пед.акселер.,%', 'Нагрузка на двигатель, %',
       'Давл.масла двиг.,кПа', 'Обор.двиг.,об/мин',
       'Значение счетчика моточасов, час:мин', 'КПП. Давление масла в системе смазки',
       'Скорость', 'ДВС. Давление смазки',
       'ДВС. Температура охлаждающей жидкости',
       'Давление в пневмостистеме (spn46), кПа', 'Уровень топлива % (spn96)',
        'ДВС. Частота вращения коленчатого вала']

cat_feat = ['iButton2','Сост.пед.сцепл.']
features = num_feat+cat_feat
len(features)

14

Посмотрим на баланс классов

In [54]:
y.sum()/y.count()

0.8069374665112325

Сделаем андерсемплинг для баланса классов увеличения качества предсказаний по меньшему по размеру классу

In [47]:
from imblearn.under_sampling import RandomUnderSampler
import pandas as pd

# Разделение на признаки и целевую переменную
X = df_anomaly[features]
y = df_anomaly['anomaly']

# Создание экземпляра RandomUnderSampler
under_sampler = RandomUnderSampler(random_state=SEED)

# Применение андерсемплинга
X_resampled, y_resampled = under_sampler.fit_resample(X, y)

In [48]:
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled,
    y_resampled,
    test_size=0.2,
    random_state=SEED
)

### Обучим модель

In [49]:
simple_model = CatBoostClassifier(iterations = 50, random_state = SEED, task_type="GPU", verbose=50,
                                    cat_features=cat_feat)
simple_model.fit(X_train, y_train)

Learning rate set to 0.360558
0:	learn: 0.4064101	total: 14.7ms	remaining: 722ms
49:	learn: 0.0005199	total: 599ms	remaining: 0us


### Метрики качества

In [50]:
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

print('Precision: ', precision_score(y_test,simple_model.predict(X_test)))
print('Recall: ', recall_score(y_test,simple_model.predict(X_test)))
print('F1: ', f1_score(y_test,simple_model.predict(X_test)))

Precision:  0.9999208171668382
Recall:  0.9998152595603178
F1:  0.9998680355776082


### Важность признаков в обучении

In [51]:
pd.DataFrame({'col': simple_model.feature_names_,
                  'importance': simple_model.get_feature_importance()}).sort_values('importance', ascending=False)

,col,importance
9,"Давление в пневмостистеме (spn46), кПа",13.698987
5,КПП. Давление масла в системе смазки,11.964382
7,ДВС. Давление смазки,10.950146
13,Сост.пед.сцепл.,9.977988
2,"Давл.масла двиг.,кПа",9.889045
8,ДВС. Температура охлаждающей жидкости,8.833770
4,"Значение счетчика моточасов, час:мин",8.217288
0,"Полож.пед.акселер.,%",7.151743
6,Скорость,7.017754
3,"Обор.двиг.,об/мин",6.315994


### Сохранение результата в виде pickle файла

In [24]:
import joblib
joblib.dump(simple_model,'anomaly_catboost.pickle')

['anomaly_catboost.pickle']